In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn import preprocessing

In [3]:
data_df = pd.read_csv(r'data.csv')

In [4]:
print(data_df.head())

   event_date activity_group reason_for_travel_detailed origin_country_code  \
0  2019-01-07        Booking       Leisure - Non-Family                  DE   
1  2019-01-16      Searching                   Business                  GB   
2  2021-01-28      Searching       Leisure - Non-Family                  DE   
3  2021-01-28      Searching       Leisure - Non-Family                  US   
4  2019-01-03      Searching                   Business                  DE   

   hotel_city hotel_state hotel_country traveler_value_group  \
0  dusseldorf         NaN            de                  Med   
1  dusseldorf         NaN            de                  Med   
2  dusseldorf         NaN            de                  Low   
3  dusseldorf         NaN            de                  Low   
4  dusseldorf         NaN            de                  Low   

  departure_checkin_date  total_number_events  
0             2019-01-25                    1  
1             2019-02-05                    

Let's make sure the key cells are not missing data

In [5]:
data_df = data_df[data_df['activity_group'].notna()]
data_df = data_df[data_df['hotel_city'].notna()]
data_df = data_df[data_df['traveler_value_group'].notna()]
data_df = data_df[data_df['event_date'].notna()]
data_df = data_df[data_df['departure_checkin_date'].notna()]
data_df = data_df[data_df['total_number_events'].notna()]

Let's view the breakdown of bookings vs searches

In [6]:
data_df['activity_group'].value_counts()

Searching    635677
Booking       89118
Name: activity_group, dtype: int64

Let's see the raw distribution of events by city

In [7]:
data_df['hotel_city'].value_counts()

london               104667
barcelona             51093
berlin                48636
madrid                38217
hamburg               32467
rome                  31173
birmingham            30269
munich                25548
edinburgh             25471
manchester            24550
milan                 22165
nice                  20849
lyon                  18795
bordeaux              16852
venice                16325
frankfurt             16233
florence              15841
marseille             15533
bristol               15282
liverpool             13576
koeln                 13294
mnchen                11547
malaga                10746
stuttgart             10339
lille                 10152
leeds                  9738
orly                   9323
hannover               8352
seville                8200
sevilla                7871
duesseldorf            7776
kln                    7701
palma de mallorca      7342
cologne                7177
firenze                4681
venezia             

Consolidate duplicates based on alternative spellings and character encodings

In [8]:
def duplicate_names(x):
    if x=='mnchen':   return 'munich'
    elif x=='firenze':   return 'florence'
    elif x=='kln': return 'koeln'
    elif x=='cologne':   return 'koeln'
    elif x=='sevilla':   return 'seville'
    elif x=='duesseldorf':   return 'dusseldorf'
    elif x=='venezia':   return 'venice'
    else:   return x
    
data_df['hotel_city'] = data_df['hotel_city'].apply(duplicate_names)
data_df['hotel_city'].value_counts()

london               104667
barcelona             51093
berlin                48636
madrid                38217
munich                37095
hamburg               32467
rome                  31173
birmingham            30269
koeln                 28172
edinburgh             25471
manchester            24550
milan                 22165
nice                  20849
venice                20639
florence              20522
lyon                  18795
bordeaux              16852
frankfurt             16233
seville               16071
marseille             15533
bristol               15282
liverpool             13576
malaga                10746
dusseldorf            10476
stuttgart             10339
lille                 10152
leeds                  9738
orly                   9323
hannover               8352
palma de mallorca      7342
Name: hotel_city, dtype: int64

Let's get aggregate activity counts for January 2019 and 2021 by city

In [9]:
dates=pd.to_datetime(data_df['event_date'])
year_df = pd.DataFrame([x.year for x in dates.tolist()],columns=['year'])
data_df2 = pd.concat((year_df,data_df),axis=1)
data_df2 = data_df2[['year', 'hotel_city','total_number_events']]
data_pivot=data_df2.groupby(['hotel_city','year'],as_index = False).sum().pivot('hotel_city','year')
data_pivot.columns = ['_'.join([str(char) for char in char_list]) for char_list in data_pivot.columns.values]
data_pivot

,total_number_events_2019,total_number_events_2021
hotel_city,,
barcelona,60818,30116
berlin,106084,15692
birmingham,44352,43247
bordeaux,36854,7785
bristol,29291,7265
dusseldorf,12221,2727
edinburgh,52829,8325
florence,9667,27402
frankfurt,26953,4799


Let's rank cities by percent change in 2021 relative to 2019

In [10]:
data_pivot['evt_pct_chg']=(data_pivot['total_number_events_2021']-data_pivot['total_number_events_2019'])/data_pivot['total_number_events_2019']
data_pivot.sort_values(by='evt_pct_chg',ascending=False)

,total_number_events_2019,total_number_events_2021,evt_pct_chg
hotel_city,,,
florence,9667,27402,1.834592
venice,12785,15978,0.249746
birmingham,44352,43247,-0.024914
palma de mallorca,5834,3728,-0.360987
nice,26280,16675,-0.365487
rome,35777,18446,-0.484417
barcelona,60818,30116,-0.504818
marseille,29257,10784,-0.631404
orly,12097,4060,-0.664380


Let's rank cities by overall interest in 2021

In [11]:
data_pivot.sort_values(by='total_number_events_2021',ascending=False)

,total_number_events_2019,total_number_events_2021,evt_pct_chg
hotel_city,,,
london,271545,57412,-0.788573
birmingham,44352,43247,-0.024914
barcelona,60818,30116,-0.504818
florence,9667,27402,1.834592
madrid,77536,19380,-0.750052
rome,35777,18446,-0.484417
nice,26280,16675,-0.365487
venice,12785,15978,0.249746
berlin,106084,15692,-0.852079


London has the highest booking and search activity, whereas Florence has the highest growth in booking and search since 2019

For robustness, let's rank based on bookings rather than bookings+searches

In [17]:
data_df_book = data_df[data_df['activity_group']=='Booking']
data_df_bk = pd.concat((year_df,data_df_book),axis=1)
data_df_bk = data_df_bk[['year', 'hotel_city','total_number_events']]
data_pivot_bk=data_df_bk.groupby(['hotel_city','year'],as_index = False).sum().pivot('hotel_city','year')
data_pivot_bk.columns = ['_'.join([str(char) for char in char_list]) for char_list in data_pivot_bk.columns.values]
data_pivot_bk

,total_number_events_2019,total_number_events_2021
hotel_city,,
barcelona,3518.0,551.0
berlin,9368.0,722.0
birmingham,4358.0,2650.0
bordeaux,4363.0,1171.0
bristol,2901.0,429.0
dusseldorf,1553.0,120.0
edinburgh,3653.0,285.0
florence,714.0,2170.0
frankfurt,2044.0,126.0


In [18]:
data_pivot_bk['evt_pct_chg']=(data_pivot_bk['total_number_events_2021']-data_pivot_bk['total_number_events_2019'])/data_pivot_bk['total_number_events_2019']
data_pivot_bk.sort_values(by='evt_pct_chg',ascending=False)

,total_number_events_2019,total_number_events_2021,evt_pct_chg
hotel_city,,,
florence,714.0,2170.0,2.039216
venice,613.0,431.0,-0.296900
birmingham,4358.0,2650.0,-0.391923
orly,1960.0,762.0,-0.611224
marseille,3157.0,1216.0,-0.614824
nice,2123.0,809.0,-0.618935
lille,2109.0,572.0,-0.728781
bordeaux,4363.0,1171.0,-0.731607
lyon,4961.0,1287.0,-0.740576


In [19]:
data_pivot_bk.sort_values(by='total_number_events_2021',ascending=False)

,total_number_events_2019,total_number_events_2021,evt_pct_chg
hotel_city,,,
birmingham,4358.0,2650.0,-0.391923
london,20621.0,2600.0,-0.873915
florence,714.0,2170.0,2.039216
lyon,4961.0,1287.0,-0.740576
marseille,3157.0,1216.0,-0.614824
bordeaux,4363.0,1171.0,-0.731607
madrid,6314.0,883.0,-0.860152
manchester,3877.0,860.0,-0.778179
nice,2123.0,809.0,-0.618935


Florence has the greatest growth in bookings, while Birmingham has the highest number of bookings in 2021